In [37]:
import requests
from bs4 import BeautifulSoup
import time 
import random
import pandas as pd
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) 
pd.set_option('display.max_rows', 999)
pd.options.mode.chained_assignment = None  # default='warn'

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'

<font size='3'>**Scraping for Poland flat prices for sale**</font>
- <small>Does an initial read of the first page to determine how many pages of offers are available.</small>
- <small>Loops through pages to gather data.</small>
- <small>Conducts some initial data cleaning.</small>

In [ ]:
# initial read to determine the amount of pages to parse

# variables state and and city should be changed accordingly in order to parse.

state = 'slaskie'
city = 'katowice'

city_get = requests.get(f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/{state}/{city}/{city}/{city}?viewType=listing',
                          headers={'User_Agent': user_agent}, verify=False)
city_content = BeautifulSoup(city_get.content)

city_pages = city_content.find_all('a', class_='eo9qioj1 css-5tvc2l edo3iif1')
page_list = []
for page in city_pages:
    page_list.append(int(page.text))
last_page = max(page_list)


wait_time = random.randint(4, 8)
print(f'Pages to parse: {last_page}')
print(f'Waiting {wait_time} seconds after initial read.')
time.sleep(wait_time)

# Create lists to store data

city_all_list = [] # price, price per sqm, rooms, size to be added, they come from the same class

city_price_list = [] 
city_price_per_sqm_list = []
city_room_list = []
city_size_list = []
city_loc_list = [] # locations will be populated here, they will come directly from the initial loop, unlike the others in the list loop.

counter = 1

for n in range(1, last_page + 1):
    print(f'parsing page {counter}...')
    city_get = requests.get(f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/{state}/{city}/{city}/{city}?viewType=listing',
                          headers={'User_Agent': user_agent}, verify=False)
    city_content = BeautifulSoup(city_get.content)
    city_ads = city_content.find_all('span', class_='css-1cyxwvy ei6hyam2')
    city_locs = city_content.find_all('p', class_='css-19dkezj e1n06ry53')
    
    for param in city_ads:
        city_all_list.append(param.text)
        
    for loc in city_locs:
        city_loc_list.append(loc.text)
        
    print(f'page {counter} parsed.')
    counter = counter + 1
    wait_time = random.randint(3, 8)
    print(f'pause: {wait_time} Sec.')
    time.sleep(wait_time)
    
print('cleaning lists...')

city_clean_list = [item.replace('\xa0', '').replace('zł', '').replace('m²', '').replace('/', '').replace('pokoje', '').replace('€', '').replace('pokój', '').replace('pokoi', '')
                     .replace('+', '') for item in city_all_list] # clear the list of 
city_clean_list = [item.strip() for item in city_clean_list]

print('looping through and creating new lists...')

for i in range(0, int(len(city_clean_list)), 4): # start with the first, skip 4 steps
    city_price_list.append(city_clean_list[i])
    
for i in range(1, int(len(city_clean_list)), 4): # start with the second, skip 4 steps
    city_price_per_sqm_list.append(city_clean_list[i])
    
for i in range(2, int(len(city_clean_list)), 4): # start with the third, skip 4 steps
    city_room_list.append(city_clean_list[i])
    
for i in range(3, int(len(city_clean_list)), 4): # start with the 4th, skip 4 steps
    city_size_list.append(city_clean_list[i])


print('parsing: complete.')


<font size='3'>**Dictionary creation and further data cleaning and conversion**</font>

In [39]:
final_dict = {
    'total_price_pln': city_price_list,
    'price_per_sqm_pln': city_price_per_sqm_list,
    'amt_of_rooms': city_room_list,
    'size_in_sqm': city_size_list,
    'loc': city_loc_list,
}

unfiltered_df = pd.DataFrame.from_dict(final_dict)
filtered_df = unfiltered_df[unfiltered_df['total_price_pln'] != 'Zapytaj o cenę']

filtered_df.drop_duplicates(keep='first')
filtered_df['area'] = filtered_df['loc'].str.split(',').str[1] # create area from the first occurrence of ',' to second occurence of ','

filtered_df['total_price_pln'] = filtered_df['total_price_pln'].str.strip()
filtered_df['price_per_sqm_pln'] = filtered_df['price_per_sqm_pln'].str.strip()
filtered_df['amt_of_rooms'] = filtered_df['amt_of_rooms'].str.strip()
filtered_df['size_in_sqm'] = filtered_df['size_in_sqm'].str.strip()
filtered_df['loc'] = filtered_df['loc'].str.strip()
filtered_df['area'] = filtered_df['area'].str.strip()

filtered_df['price_per_sqm_pln'] = filtered_df['price_per_sqm_pln'].str.replace(',', '.')
filtered_df['total_price_pln'] = filtered_df['total_price_pln'].str.replace(',', '.')
filtered_df['size_in_sqm'] = filtered_df['size_in_sqm'].str.replace(',', '.')

convert_dict = {
    'price_per_sqm_pln': 'float',
    'total_price_pln': 'float',
    'size_in_sqm': 'float',
    'amt_of_rooms': 'int'
}

filtered_df = filtered_df.astype(convert_dict)

filtered_df.info()
filtered_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3654 entries, 1 to 4016
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   total_price_pln    3654 non-null   float64
 1   price_per_sqm_pln  3654 non-null   float64
 2   amt_of_rooms       3654 non-null   int32  
 3   size_in_sqm        3654 non-null   float64
 4   loc                3654 non-null   object 
 5   area               3654 non-null   object 
dtypes: float64(3), int32(1), object(2)
memory usage: 185.6+ KB


,total_price_pln,price_per_sqm_pln,amt_of_rooms,size_in_sqm,loc,area
1,225000.0,6018.0,2,37.39,"al. gen. Józefa Hallera, Szopienice-Burowiec, ...",Szopienice-Burowiec
2,425616.0,10055.0,2,42.33,"al. Wojciecha Korfantego, Koszutka, Katowice, ...",Koszutka
3,315000.0,7393.0,1,42.61,"ul. 3 Maja 11, Śródmieście, Katowice, śląskie",Śródmieście
4,314900.0,4998.0,2,63.00,"ul. Pawła Chromika, Zawodzie, Katowice, śląskie",Zawodzie
5,420000.0,9745.0,2,43.10,"ul. Bytkowska, Wełnowiec-Józefowiec, Katowice,...",Wełnowiec-Józefowiec


In [40]:
filtered_df.to_excel(f'files/housing_data_flats_{city}.xlsx', index=False)